# PLIPify
#### Exploring SARS-CoV-2 Main Protease Interaction Hotspots Using a Novel Protein-Ligand Interaction Frequency Fingerprint
With the COVID-19 pandemic caused by the severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) becoming a global health emergency, there is an urgent need to develop effective anti-COVID drugs to combat this virus. The main protease of SARS-CoV-2 MPro appears to be a promising target. However more information on how compounds bind to MPro is needed to effectively target it.

Protein-ligand interactions are and essential part of drug discovery. While tools exist to easily identify interactions of single structures, this project aims to create an interaction frequency profile across multiple structures of the same protein. To this end we will analyze a set of co-crystallized fragment structures of MPro with a tool named PLIP and will calculate and visualize a count-based fingerprint across all these structures.

While the focus of this notebook in MPro, this pipeline can be used for any set of protein structures that meet the requirements.

<img src="./data/interactionpic.PNG">

## Aim of this notebook

**Key question:** How do compounds bind to MPro?
* Gain insight on protein-ligand interactions
* Generate profile from multiple structures of the same protein
* Create interaction count fingerprint based on these structures
* Possibly find interaction hotspots of MPro
* Make pipline applicable to other proteins


## Contents
### Background
* PLIP
* Molecular Fingerprints
    * Interaction Fingerprints
    * Cumulative Interaction Fingerprint
* Module Structure

### Practical
* Data
* Fingerprint calculation
* Fingerprint Visualization
    * Bar Plot and Heatmap
    * Fingerprint Table
* Results
* Outlook and questions

## PLIP
The <a href="https://projects.biotec.tu-dresden.de/plip-web/plip/index">Protein-Ligand Interaction Profiler (PLIP)</a> is a tool that idenitifies noncovalent interactions between proteins and their ligands. It is available as web tool and a python command-line application and was create by the Michael Schroeder group at the Biotechnology Center TU Dresden.
PLIP takes a single PDB file or PDB ID to create and interaction report. It uses four steps to detect and report relevant interactions: structure preparation, functional characterization, rule-based matching and filtering of interactions.

* Reports interactions for an uploaded PDB file or PDB ID
* Detailed interaction report
* No structure preparation required

#### Interaction Types
* hydrophobic
* hbond
* waterbridge
* saltbridge
* pistacking
* pication
* halogen
* metal

#### Citation
Salentin,S. et al. PLIP: fully automated protein-ligand interaction profiler. Nucl. Acids Res. (1 July 2015) 43 (W1): W443-W447. doi: 10.1093/nar/gkv315

## Molecular Fingerprints
Molecular fingerprints are a computational representation of molecules that encode chemical and molecular features in form of bitstrings, bitvectors or arrays. Each bit corresponds to a predefined molecular feature or environment, which in this case is a specific interaction type and residue combination. Most commonly in fingerprints each bit contain either a "1" representing the presence and "0" the absence of a feature. However some implementations are count-based, where each bit counts how often a specific feature is present.

### Interaction Fingerprints
Interaction fingerprints are a form of molecular fingerprint in which, the features represented by each bit are a unique combination of residues and interaction types. Each residue will have the same number of bits assigned to it in the fingerprint. Each bit of a residue represents one interaction type. The order of the defined interaction types are the same for each residue. As previously mentioned, a bit set to "1" indicated the presence and "0" the absence of an interaction. For count-based implementation, each bit is set to the specific number of interactions that were found. The length of the fingerprint is the number of pre-defined residues multiplied by the defined interaction types.

<img src="./data/ifps_1.png">

#### PLIPify specific
* count-based
* 8 interaction types
* pre-defined residues depend on structure
* each residue has 8 bits, one per interaction type

### Cumulative Interaction Fingerprint
As the main purpose of this project is to analyze interactions across multiple structures of the same protein and possibly find interaction hotspots, we don't only need single structure fingerprints, but a fingerprint that reports on interactions across all given structures.
For this purpose, a cumulative fingerprint is created by summing up all the single structure fingerprints.

<img src="./data/plipify_fps.png">

#### Pre-requisites
* All structures must be of the same protein
* Structures must have the same residue numbering
* Structures with non-covalent ligands
* Pre-defined residues must be provided

## Module Structure
For this project multiple python modules were created containing classes and functions that are used throught this notebook. The following diagram shows a simplified overview of what each module is responsible for.
<img src ="./data/plipify_modules.PNG">

# Pratical
The following steps will calculate and visualize a cumulative interaction fingerprint based on provided structures and pre-defined residues.

## Data

> To contribute to the global effort to combat COVID-19, Diamond has been able to solve a new structure of the SARS-CoV-2 main protease (MPro) at high resolution (PDB ID: 6YB7), and complete a large XChem crystallographic fragment screen against it (detailed below). Data have been deposited with the PDB, but we are making the results available immediately to the world on this page; additional work is ongoing, and updates will be continually posted here in coming days and weeks.

https://www.diamond.ac.uk/covid-19/for-scientists/Main-protease-structure-and-XChem.html 

## Fingerprint Calculation

In this section, the PLIPify interaction fingerprint is created as described before. The interaction fingerprints for individual structures are count based fingerprints which's length depends on the number of pre-defined residues and the number of interaction types. PLIP provides data for eight interaction types, so each residue is represented with eight bits, one for each type, in the fingerprint. When multiple structures of the same protein are analyzed, the indiviual fingerprints are summed up and a frequency value is calculated for each bit.

The calculated fingerprint can be returned as:
* dataframes with additional information
* simple array for further computing

Additionally, for the dataframe there is the option to delete all residues that have no interactions with any interaction type.

#### Import neccessary packages

In [2]:
from plip_fingerprints import read_residues
from core import Structure
from fingerprints import InteractionFingerprint
import os
import pandas as pd
from tqdm.notebook import tqdm

### Parameters
- structure_folder = Folder containing the protein structures
- name_file = File containing the names of the structures in folder
- residue_file = Pre-defined binding site residues as a csv file with ','-delimited residue numbers

In [12]:
structure_folder = "./data/diamond_xchem_screen_mpro_all_pdbs"
name_file = "diamond_xchem_screen_mpro_non_covalent_pdbs.dat"
residue_file = './data/MPro_residues.csv'

The pre-defined residues of MPro were obtained through the <a href="https://github.com/volkamerlab/covid19-SBapproach/blob/master/code/focused_library_similar_proteins/01_binding_site_definition.ipynb">binding side definition notebook.</a>

### Fingerprint main function
This function generates the count- and frequency-based fingerprint over all given structures.

In [4]:
indices = [int(x) for x in pd.read_csv(residue_file).columns]

def calculate_fingerprints(structure_folder, name_file, residue_file):
    with open(os.path.join(structure_folder, name_file)) as f:
        non_covalent_filenames = [l.strip() for l in f if l.strip()]
    structures = []
    for filename in tqdm(non_covalent_filenames):
        full_filename = os.path.join(structure_folder, filename)
        if not os.path.isfile(full_filename):
            print("File", full_filename, "does not exist?")
            continue
        structures.append(Structure.from_pdbfile(full_filename))
    fp_creator = InteractionFingerprint(structures, indices)
    
    # count fps
    count_fp = fp_creator.calculate_fingerprint(labeled=True, as_dataframe=True, remove_non_interacting_residues=True)
    count_fp_all = fp_creator.calculate_fingerprint(labeled=True, as_dataframe=True, remove_non_interacting_residues=False)
    
    # frequency fps
    freq_fp = count_fp.div(len(structures))
    freq_fp = freq_fp.round(decimals=2)
    freq_fp_all = count_fp_all.div(len(structures))
    freq_fp_all = freq_fp_all.round(decimals=2)
    return count_fp, count_fp_all, freq_fp, freq_fp_all

### MPro Interaction Fingerprint
The fingerprint calculation for the provided MPro Structures.
* 68 pre-defined residues
* 8 Interaction Types

In [5]:
count_fp, count_fp_all, freq_fp, freq_fp_all = calculate_fingerprints(structure_folder, name_file, residue_file)

('File', './data/diamond_xchem_screen_mpro_all_pdbs/Mpro-x0397.pdb', 'does not exist?')
('File', './data/diamond_xchem_screen_mpro_all_pdbs/Mpro-x0426.pdb', 'does not exist?')
('File', './data/diamond_xchem_screen_mpro_all_pdbs/Mpro-x0395.pdb', 'does not exist?')



In [7]:
count_fp

,halogen,hbond,hydrophobic,metal,pication,pistacking,saltbridge,waterbridge
THR:25.A,0,2,2,0,0,0,0,0
HIS:41.A,0,3,2,0,0,9,1,0
THR:45.A,0,1,0,0,0,0,0,0
MET:49.A,0,0,1,0,0,0,0,0
ASN:142.A,0,3,0,0,0,0,0,1
SER:144.A,0,1,0,0,0,0,0,0
HIS:163.A,0,1,0,0,0,0,0,0
HIS:164.A,0,0,0,0,0,0,0,2
MET:165.A,0,0,5,0,0,0,0,0
GLU:166.A,0,13,5,0,0,0,0,4


## Fingerprint Visualization
After having calculated the cumulative interaction fingerprint for MPro, the results need to be put in a form in which they are easily readable. Hence, multiple forms of visualization have been made available. Based on the user's the selection, the results from the fingerprint can be displayed in table of plot form. Multiple additional selections can be made for the indivual visualizations.

### Stacked Bar Chart and Heatmap
This section visualizes the fingerprint as a Stacked Bar Chart or Heatmap based on the user's preference.

#### Parameter Selection
The fingerprint can displayed as a bar chart or heatmap, based on the user's selection. Additionally, the visualization can be based on the count or frequency based fingerprint.
#### Bar Chart
Interactive: Hover over the bars of interest to see the specific number of interactions for the residue and interaction type of interest.


In [8]:
from fp_visual import fingerprint_barplot, fingerprint_heatmap
import ipywidgets as widgets
from ipywidgets import HBox
from IPython.display import display, clear_output

In [9]:
plot_choice = widgets.Dropdown(
    options=['bar','heatmap'],
    value='bar',
    description='Plot Type:',
    disabled=False,
)
fp_choice_plot = widgets.Dropdown(
    options=['count','frequency'],
    value='count',
    description='Fingerprint:',
    disabled=False,
)
        
button = widgets.Button(description="Display Plot!")

def on_button_clicked(b):
    """
    Display the fingerprint plot on click based on the user selections.
    """
    clear_output()
    display(HBox([plot_choice, fp_choice_plot, button]))
    if (fp_choice_plot.value  == 'count') and (plot_choice.value == 'bar'):
        fingerprint_barplot(count_fp)
    elif (fp_choice_plot.value  == "frequency") and (plot_choice.value == 'bar'):
        fingerprint_barplot(freq_fp)
    elif (fp_choice_plot.value  == "count") and (plot_choice.value == "heatmap"):
        fingerprint_heatmap(count_fp)
    else:
        fingerprint_heatmap(freq_fp)
        
button.on_click(on_button_clicked)

HBox([plot_choice, fp_choice_plot, button])

SEJveChjaGlsZHJlbj0oRHJvcGRvd24oZGVzY3JpcHRpb249dSdQbG90IFR5cGU6Jywgb3B0aW9ucz0oJ2JhcicsICdoZWF0bWFwJyksIHZhbHVlPSdiYXInKSwgRHJvcGRvd24oZGVzY3JpcHTigKY=


### Fingerprint Table
In this section the interaction fingerprint can viewd in tabular form. Once again this can be based on either the count-based or the frequency-based fingerprint. Additionally, it is possible to show the table for all pre-defined residues or only residues which have at least one interaction.
The table shows the number or frequency of interactions for each residue. The types of interactions are colour coded.

In [10]:
from fp_visual import fingerprint_table

In [11]:
table_choice = widgets.Dropdown(
    options=['full table','only non empty cells'],
    value='full table',
    description='Table View:',
    disabled=False,
)
fp_choice = widgets.Dropdown(
    options=['count','frequency'],
    value='count',
    description='Fingerprint:',
    disabled=False,
)
tbutton = widgets.Button(description="Display Table!")

def on_tbutton_clicked(b):
    """
    Display the fingerprint table on click based on the user selections.
    """
    clear_output()
    display(HBox([table_choice, fp_choice, tbutton]))
    if (fp_choice.value  == 'count') and (table_choice.value == 'full table'):
        fingerprint_table(count_fp_all)
    elif (fp_choice.value  == 'frequency') and (table_choice.value == 'full table'):
        fingerprint_table(freq_fp_all)
    elif (fp_choice.value  == 'count') and (table_choice.value == 'only non empty cells'):
        fingerprint_table(count_fp)
    else:
        fingerprint_table(freq_fp)
        
tbutton.on_click(on_tbutton_clicked)

HBox([table_choice, fp_choice, tbutton])

SEJveChjaGlsZHJlbj0oRHJvcGRvd24oZGVzY3JpcHRpb249dSdUYWJsZSBWaWV3OicsIGluZGV4PTEsIG9wdGlvbnM9KCdmdWxsIHRhYmxlJywgJ29ubHkgbm9uIGVtcHR5IGNlbGxzJyksIHbigKY=


waterbridge,halogen,saltbridge,hbond,pistacking,metal,hydrophobic,pication


## Results

While analyzing the MPro fragments a total of <b>71</b> protein-ligand interactions were found in <b>13</b> distinct residues were found.

The highest number of interactions were found interactions were found in 
* residue <b>GLU:166.A</b> has 22 interactions -> 13 hbond
* residue <b>HIS:41.A</b> has 14 interactions -> 9 pistacking
* residue <b>GLU:166.A</b> has 13 interactions -> 9 hydrophic

## Outlook and open ends
While insights can be gathered from this project in it's current state, there are multiple possibilities for improvements and additions.
* Code optimization
* Tweaking/Adding to 2D Visualization
* Manual colour selection for visuals
* Add more residue/interaction information from the PLIP report
* Link to interaction detail in table
* 3D Visualization
* Python 3 PLIP update
* Multiple interactions of the same type for one residue?

### Optional: Show all data acquired from PLIP in Data Frame.

In [1]:
from plip_fingerprints import get_plip_data, show_plip_data
interactions = get_plip_data("./data/diamond_xchem_screen_mpro_all_pdbs","diamond_xchem_screen_mpro_non_covalent_pdbs.dat")
show_plip_data(interactions)

('File', './data/diamond_xchem_screen_mpro_all_pdbs/Mpro-x0397.pdb', 'does not exist?')
('File', './data/diamond_xchem_screen_mpro_all_pdbs/Mpro-x0426.pdb', 'does not exist?')
('File', './data/diamond_xchem_screen_mpro_all_pdbs/Mpro-x0395.pdb', 'does not exist?')



# Structure Mpro-x0946.pdb

## Site LIG:A:1101

### waterbridge

,RESNR,RESTYPE,RESCHAIN,RESNR_LIG,RESTYPE_LIG,RESCHAIN_LIG,DIST_A-W,DIST_D-W,DON_ANGLE,WATER_ANGLE,PROTISDON,DONOR_IDX,DONORTYPE,ACCEPTOR_IDX,ACCEPTORTYPE,WATER_IDX,LIGCOO,PROTCOO,WATERCOO
0,166,GLU,A,1101,LIG,A,4.03,3.11,173.92,93.79,True,1288,Nam,2383,N3,2434,"(9.554, 5.104, 22.977)","(9.904, 2.716, 18.584)","(9.307, 1.422, 21.348)"


...

,RESNR,RESTYPE,RESCHAIN,RESNR_LIG,RESTYPE_LIG,RESCHAIN_LIG,SIDECHAIN,DIST_H-A,DIST_D-A,DON_ANGLE,PROTISDON,DONORIDX,DONORTYPE,ACCEPTORIDX,ACCEPTORTYPE,LIGCOO,PROTCOO
0,41,HIS,A,1101,LIG,A,True,2.44,3.08,122.60,True,309,Nar,2444,N1,"(14.061, -1.841, 20.54)","(12.794, -4.607, 21.042)"


### pistacking

,RESNR,RESTYPE,RESCHAIN,RESNR_LIG,RESTYPE_LIG,RESCHAIN_LIG,CENTDIST,ANGLE,OFFSET,TYPE,LIG_IDX_LIST,LIGCOO,PROTCOO
0,41,HIS,A,1101,LIG,A,5.14,87.37,1.26,T,"2436,2437,2438,2439,2441,2442","(11.690833333333332, -0.2608333333333333, 23.2...","(11.795599999999999, -5.0338, 21.373)"


### hydrophobic

,RESNR,RESTYPE,RESCHAIN,RESNR_LIG,RESTYPE_LIG,RESCHAIN_LIG,DIST,LIGCARBONIDX,PROTCARBONIDX,LIGCOO,PROTCOO
0,189,GLN,A,1101,LIG,A,3.85,2437,1482,"(12.014, 0.979, 23.834)","(12.876, 2.521, 27.25)"
